In [1]:
import numpy as np
import pandas as pd
import math
import datetime
from datetime import datetime
import pickle
from scipy import stats

from pickle_file_config import *
from previous_utils.utils import *

# US Data:

In [ ]:
# Preprocess Test:
us_bi_directional_path = US_HRNN_PATH

us_bi_directional_data=preprocess_data(us_bi_directional_path, 12, 1)
us_bi_directional_data.columns = columns_order
us_bi_directional_data.sort_values(['Category','Category_id','Date'], inplace = True)
us_bi_directional_dataset_dict = {}

for category in list(us_bi_directional_data['Category'].unique()):
    us_bi_directional_dataset_dict[category] = us_bi_directional_data[us_bi_directional_data['Category'] == category]
    
with open(US_bi_directional_pickle, 'wb') as handle: #wb means write - creating a pickle file
    pickle.dump(us_bi_directional_dataset_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Sanity Test:
us_bi_directional_dataset_dict['All items']

## Creating US's (1) Son-Parent Dict, (2) Category ID to Name Dict, (3) Categories per Indent Dict, (4) Parent to Sons List Dict:

In [ ]:
us_parent_dict = {}
us_weight_dict = {}
us_cat_cat_id_dict = {}
us_indent_dict = {}
us_parent_to_sons_dict = {}

us_parent_df = us_bi_directional_data[['Category_id', 'Parent_ID']].drop_duplicates()
us_cat_weight_df = us_bi_directional_data[['Category_id','Weight']].drop_duplicates()
us_cat_cat_id_df = us_bi_directional_data[['Category_id','Category']].drop_duplicates()
us_indent_df = us_bi_directional_data[['Indent','Category_id']].drop_duplicates()

for category_id in list(us_parent_df['Category_id'].unique()):
    us_parent_dict[category_id] = us_parent_df[us_parent_df['Category_id'] == category_id]['Parent_ID'].values[0]
    
for category_id in list(us_cat_weight_df['Category_id'].unique()):
    us_weight_dict[category_id] = us_cat_weight_df[us_cat_weight_df['Category_id'] == category_id]['Weight'].values[0]
    
for category_id in list(us_cat_cat_id_df['Category_id'].unique()):
    us_cat_cat_id_dict[category_id] = us_cat_cat_id_df[us_cat_cat_id_df['Category_id'] == category_id]['Category'].values[0]
    
for indent in list(us_indent_df['Indent'].unique()):
    us_indent_dict[indent] = list(us_indent_df[us_indent_df['Indent']==indent].Category_id.unique())

for cat_id in list(us_bi_directional_data.Category_id.unique()):
    if us_parent_dict[cat_id] in list(us_parent_to_sons_dict.keys()):
        us_parent_to_sons_dict[us_parent_dict[cat_id]]['sons'].append(cat_id)
    
    else:
        us_parent_to_sons_dict[us_parent_dict[cat_id]] = {'sons': [cat_id]}


In [ ]:
with open(us_parent_son_pickle, 'wb') as handle: 
    pickle.dump(us_parent_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(us_cat_weight_pickle, 'wb') as handle: 
    pickle.dump(us_weight_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open(us_category_id_to_name_pickle, 'wb') as handle: 
    pickle.dump(us_cat_cat_id_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(us_category_id_per_indent_pickle, 'wb') as handle: #wb means write - creating a pickle file
    pickle.dump(us_indent_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(us_parent_to_sons_list_pickle, 'wb') as handle: #wb means write - creating a pickle file
    pickle.dump(us_parent_to_sons_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# print categories with no parents - making sure the only category without parent id is all items (8106)
for cat_id in list(us_parent_dict.keys()):
    if pd.isna(us_parent_dict[cat_id]):
        print(cat_id)

Creating Parent Son Name Dict:

In [ ]:
with open(us_category_id_to_name_pickle, 'rb') as f:
    category_id_to_name_dict = pickle.load(f)

with open(us_parent_son_pickle, 'rb') as f:
    son_parent_dict = pickle.load(f)

parent_son_name_dict = {}

for son_id in list(category_id_to_name_dict.keys()):
    son_id = int(son_id)
    son_name = category_id_to_name_dict[son_id]
    if son_name == 'All items':
        parent_son_name_dict[son_name] = None
    else:
        parent_id = int(son_parent_dict[son_id])
        if parent_id in list(category_id_to_name_dict.keys()):
            parent_name = category_id_to_name_dict[parent_id]
            parent_son_name_dict[son_name] = parent_name

with open(us_parent_son_name_pickle, 'wb') as handle: #wb means write - creating a pickle file
    pickle.dump(parent_son_name_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

#  Canada Data:

In [ ]:
# Preprocess Test:
canada_bi_directional_path = CANADA_HRNN_PATH

canada_bi_directional_data=preprocess_data(canada_bi_directional_path, 12, 1)
canada_bi_directional_data.columns = columns_order
canada_bi_directional_data.sort_values(['Category','Category_id','Date'], inplace = True)
canada_bi_directional_dataset_dict = {}

for category in list(canada_bi_directional_data['Category'].unique()):
    canada_bi_directional_dataset_dict[category] = canada_bi_directional_data[canada_bi_directional_data['Category'] == category]
    
with open(canada_bi_directional_pickle, 'wb') as handle: #wb means write - creating a pickle file
    pickle.dump(canada_bi_directional_dataset_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Sanity Test:
canada_bi_directional_dataset_dict['All-items']

## Creating Canada's (1) Son-Parent Dict, (2) Category ID to Name Dict, (3) Categories per Indent Dict, (4) Parent to Sons List Dict:

In [ ]:
canada_parent_dict = {}
canada_weight_dict = {}
canada_cat_cat_id_dict = {}
canada_indent_dict = {}
canada_parent_to_sons_dict = {}

canada_parent_df = canada_bi_directional_data[['Category_id', 'Parent_ID']].drop_duplicates()
canada_cat_weight_df = canada_bi_directional_data[['Category_id','Weight']].drop_duplicates()
canada_cat_cat_id_df = canada_bi_directional_data[['Category_id','Category']].drop_duplicates()
canada_indent_df = canada_bi_directional_data[['Indent','Category_id']].drop_duplicates()

for category_id in list(canada_parent_df['Category_id'].unique()):
    canada_parent_dict[category_id] = canada_parent_df[canada_parent_df['Category_id'] == category_id]['Parent_ID'].values[0]
    
for category_id in list(canada_cat_weight_df['Category_id'].unique()):
    canada_weight_dict[category_id] = canada_cat_weight_df[canada_cat_weight_df['Category_id'] == category_id]['Weight'].values[0]
    
for category_id in list(canada_cat_cat_id_df['Category_id'].unique()):
    canada_cat_cat_id_dict[category_id] = canada_cat_cat_id_df[canada_cat_cat_id_df['Category_id'] == category_id]['Category'].values[0]
    
for indent in list(canada_indent_df['Indent'].unique()):
    canada_indent_dict[indent] = list(canada_indent_df[canada_indent_df['Indent']==indent].Category_id.unique())

for cat_id in list(canada_bi_directional_data.Category_id.unique()):
    if canada_parent_dict[cat_id] in list(canada_parent_to_sons_dict.keys()):
        canada_parent_to_sons_dict[canada_parent_dict[cat_id]]['sons'].append(cat_id)
    
    else:
        canada_parent_to_sons_dict[canada_parent_dict[cat_id]] = {'sons': [cat_id]}


In [ ]:
with open(canada_parent_son_pickle, 'wb') as handle: 
    pickle.dump(canada_parent_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(canada_cat_weight_pickle, 'wb') as handle: 
    pickle.dump(canada_weight_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open(canada_category_id_to_name_pickle, 'wb') as handle: 
    pickle.dump(canada_cat_cat_id_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(canada_category_id_per_indent_pickle, 'wb') as handle: #wb means write - creating a pickle file
    pickle.dump(canada_indent_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(canada_parent_to_sons_list_pickle, 'wb') as handle: #wb means write - creating a pickle file
    pickle.dump(canada_parent_to_sons_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# print categories with no parents - making sure the only category without parent id is all items (2)
for cat_id in list(canada_parent_dict.keys()):
    if pd.isna(canada_parent_dict[cat_id]):
        print(cat_id)

Creating Parent Son Name Dict:

In [ ]:
with open(canada_category_id_to_name_pickle, 'rb') as f:
    category_id_to_name_dict = pickle.load(f)

with open(canada_parent_son_pickle, 'rb') as f:
    son_parent_dict = pickle.load(f)

parent_son_name_dict = {}

for son_id in list(category_id_to_name_dict.keys()):
    son_id = int(son_id)
    son_name = category_id_to_name_dict[son_id]
    if son_name == 'All-items':
        parent_son_name_dict[son_name] = None
    else:
        parent_id = int(son_parent_dict[son_id])
        if parent_id in list(category_id_to_name_dict.keys()):
            parent_name = category_id_to_name_dict[parent_id]
            parent_son_name_dict[son_name] = parent_name

with open(canada_parent_son_name_pickle, 'wb') as handle: #wb means write - creating a pickle file
    pickle.dump(parent_son_name_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Norway Data:

In [ ]:
# Preprocess Test:
norway_bi_directional_path = NORWAY_HRNN_PATH

norway_bi_directional_data=preprocess_data(norway_bi_directional_path, 12, 1)
norway_bi_directional_data.columns = columns_order
norway_bi_directional_data.sort_values(['Category','Category_id','Date'], inplace = True)
norway_bi_directional_dataset_dict = {}

for category in list(norway_bi_directional_data['Category'].unique()):
    norway_bi_directional_dataset_dict[category] = norway_bi_directional_data[norway_bi_directional_data['Category'] == category]
    
with open(norway_bi_directional_pickle, 'wb') as handle: #wb means write - creating a pickle file
    pickle.dump(norway_bi_directional_dataset_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Sanity Test:
norway_bi_directional_dataset_dict['All-items']

## Creating Norway's (1) Son-Parent Dict, (2) Category ID to Name Dict, (3) Categories per Indent Dict, (4) Parent to Sons List Dict:

In [ ]:
norway_parent_dict = {}
norway_weight_dict = {}
norway_cat_cat_id_dict = {}
norway_indent_dict = {}
norway_parent_to_sons_dict = {}

norway_parent_df = norway_bi_directional_data[['Category_id', 'Parent_ID']].drop_duplicates()
norway_cat_weight_df = norway_bi_directional_data[['Category_id','Weight']].drop_duplicates()
norway_cat_cat_id_df = norway_bi_directional_data[['Category_id','Category']].drop_duplicates()
norway_indent_df = norway_bi_directional_data[['Indent','Category_id']].drop_duplicates()

for category_id in list(norway_parent_df['Category_id'].unique()):
    norway_parent_dict[category_id] = norway_parent_df[norway_parent_df['Category_id'] == category_id]['Parent_ID'].values[0]
    
for category_id in list(norway_cat_weight_df['Category_id'].unique()):
    norway_weight_dict[category_id] = norway_cat_weight_df[norway_cat_weight_df['Category_id'] == category_id]['Weight'].values[0]
    
for category_id in list(norway_cat_cat_id_df['Category_id'].unique()):
    norway_cat_cat_id_dict[category_id] = norway_cat_cat_id_df[norway_cat_cat_id_df['Category_id'] == category_id]['Category'].values[0]
    
for indent in list(norway_indent_df['Indent'].unique()):
    norway_indent_dict[indent] = list(norway_indent_df[norway_indent_df['Indent']==indent].Category_id.unique())

for cat_id in list(norway_bi_directional_data.Category_id.unique()):
    if norway_parent_dict[cat_id] in list(norway_parent_to_sons_dict.keys()):
        norway_parent_to_sons_dict[norway_parent_dict[cat_id]]['sons'].append(cat_id)
    
    else:
        norway_parent_to_sons_dict[norway_parent_dict[cat_id]] = {'sons': [cat_id]}


In [ ]:
with open(norway_parent_son_pickle, 'wb') as handle: 
    pickle.dump(norway_parent_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(norway_cat_weight_pickle, 'wb') as handle: 
    pickle.dump(norway_weight_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open(norway_category_id_to_name_pickle, 'wb') as handle: 
    pickle.dump(norway_cat_cat_id_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(norway_category_id_per_indent_pickle, 'wb') as handle: #wb means write - creating a pickle file
    pickle.dump(norway_indent_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(norway_parent_to_sons_list_pickle, 'wb') as handle: #wb means write - creating a pickle file
    pickle.dump(norway_parent_to_sons_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# print categories with no parents - making sure the only category without parent id is all items (0)
for cat_id in list(norway_parent_dict.keys()):
    if pd.isna(norway_parent_dict[cat_id]):
        print(cat_id)

Creating Parent Son Name Dict:

In [ ]:
with open(norway_category_id_to_name_pickle, 'rb') as f:
    category_id_to_name_dict = pickle.load(f)

with open(norway_parent_son_pickle, 'rb') as f:
    son_parent_dict = pickle.load(f)

parent_son_name_dict = {}

for son_id in list(category_id_to_name_dict.keys()):
    son_id = int(son_id)
    son_name = category_id_to_name_dict[son_id]
    if son_name == 'All-items':
        parent_son_name_dict[son_name] = None
    else:
        parent_id = int(son_parent_dict[son_id])
        if parent_id in list(category_id_to_name_dict.keys()):
            parent_name = category_id_to_name_dict[parent_id]
            parent_son_name_dict[son_name] = parent_name

with open(norway_parent_son_name_pickle, 'wb') as handle: #wb means write - creating a pickle file
    pickle.dump(parent_son_name_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)